 # Predicting Book Success

In [1]:
#Import Dependencies
from path import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf



In [2]:

#Import the dataset
data = Path('books_clean.csv')
df = pd.read_csv(data)

# We can see from the preview of the DataFrame that multiple variables (also called features), such as the isbn13,
#published_year, average_rating, num_pages, ratings_count, can be used to predict the outcome: whether a book will have a good 
#rating (1) or will not (0) based on the fact that an average rating below of 4.5 will not likely be successful

df.loc[df['Rating'] <= 4.1, 'Rating_Classification'] = 'Low_Rating' 
df.loc[df['Rating'] > 4.1, 'Rating_Classification'] = 'High_Rating'

df.reset_index(inplace=True, drop=True)
df.head()

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Rating_Classification
0,9780439358071,4.49,7.38,870,1996446,English,Big-Publisher,Mid-Fame,Top_Category,Serie,High_Rating
1,9780316015844,3.59,2.10,498,4367341,English,Big-Publisher,Low-Fame,Other,Serie,Low_Rating
2,9780345538376,4.59,21.15,1728,97731,English,Big-Publisher,High-Fame,Top_Category,Serie,High_Rating
3,9780393978896,3.84,2.73,464,1041597,English,Big-Publisher,Low-Fame,Top_Category,Other,Low_Rating
4,9780142437209,4.11,5.46,532,1328143,English,Big-Publisher,Low-Fame,Top_Category,Other,High_Rating


In [3]:
# We will check for the variables from all columns
df.dtypes

ISBN                       int64
Rating                   float64
Price                    float64
Pages                      int64
Rating Count               int64
Language_Dummy            object
Size_Dummy                object
Famous_Dummy              object
Categories_Dummy          object
Serie_Dummy               object
Rating_Classification     object
dtype: object

In [4]:
#Count the current NaN values from the dataframe
df.isnull().sum().sum()

0

In [5]:
#Drop all the NaN values
df.dropna(inplace=True)

In [6]:
#Recount all the NaN values to make sure they are dropped
df.isnull().sum().sum()

0

In [7]:
#Use method to convert String to int
def rating(x):
    if x == 'Low_Rating':
        return 0
    if x == 'High_Rating':
        return 1
    

In [8]:
#Apply the method to the rating_classification column
df['Rating_Classification'] = df['Rating_Classification'].apply(rating)

In [9]:
#Use method to convert String to int
df=pd.get_dummies(df)

In [10]:
#Create a new dataframe with the target variables
df.tail(5)

,ISBN,Rating,Price,Pages,Rating Count,Rating_Classification,Language_Dummy_English,Language_Dummy_Other,Size_Dummy_Big-Publisher,Size_Dummy_Mid-Publisher,Size_Dummy_Small-Publisher,Famous_Dummy_High-Fame,Famous_Dummy_Low-Fame,Famous_Dummy_Mid-Fame,Categories_Dummy_Other,Categories_Dummy_Top_Category,Serie_Dummy_Other,Serie_Dummy_Serie
1926,9780441731183,3.78,3.89,341,6092,0,1,0,1,0,0,0,0,1,0,1,0,1
1927,9780811213769,4.30,3.59,99,599,1,1,0,1,0,0,0,1,0,0,1,1,0
1928,9780553562606,4.08,3.12,518,5356,0,1,0,1,0,0,0,1,0,0,1,0,1
1929,9781400044870,4.02,3.59,320,4455,0,1,0,1,0,0,0,1,0,0,1,1,0
1930,9780553273861,3.64,3.52,291,580,0,1,0,1,0,0,0,0,1,0,1,1,0


##  Separate the Features (X) from the Target (y)

In [11]:
#The Outcome column is defined as y, or the target.
#X, or features, is created by dropping the Outcome column from the DataFrame.

y = df["Rating_Classification"]
X = df.drop(columns=["Rating_Classification","ISBN","Rating","Rating Count"])



 ## Split our data into training and testing

In [12]:
#We first split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape


(1448, 14)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
#Examining the shape of the training set with X_train.shape returned (1636,5), meaning that there are 1636 samples (rows) and 
#five features (columns).

In [15]:
#The next step was to create a logistic regression model with the specified arguments for solver, max_iter, and random_state
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [16]:
#we trained the model with the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [17]:
#To create predictions for y-values, we used the X_test set
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.tail(20)


,Prediction,Actual
463,0,1
464,0,0
465,0,1
466,0,0
467,0,1
468,0,0
469,0,0
470,0,0
471,0,0
472,0,0


In [18]:
#The final step is to measure the accuracy of the logistic regression model created

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.6997929606625258


In [19]:
#taking into account that the accuracy score is simply the percentage of predictions that are correct. In this case, 
#the model's accuracy score was 0.9834, meaning that the model was correct 98.34% of the time.

In [20]:
#  import the relevant modules for validation and print the confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,313,15
Actual 1,130,25


In [21]:
# Report of sensitivity, precission and F1. La línea 0 se aplica al predictor de diabetes como hipótesis alternativa
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.71      0.95      0.81       328
           1       0.62      0.16      0.26       155

    accuracy                           0.70       483
   macro avg       0.67      0.56      0.53       483
weighted avg       0.68      0.70      0.63       483



# Single layer

In [22]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=len(X.columns)))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
46/46 [==============================] - 1s 2ms/step - loss: 0.7929 - accuracy: 0.6305
Epoch 2/200
46/46 [==============================] - 0s 2ms/step - loss: 0.7609 - accuracy: 0.6381
Epoch 3/200
46/46 [==============================] - 0s 2ms/step - loss: 0.7360 - accuracy: 0.6450
Epoch 4/200
46/46 [==============================] - 0s 2ms/step - loss: 0.7146 - accuracy: 0.6630
Epoch 5/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6976 - accuracy: 0.6782
Epoch 6/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6838 - accuracy: 0.6823
Epoch 7/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6721 - accuracy: 0.6927
Epoch 8/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6623 - accuracy: 0.6913
Epoch 9/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6541 - accuracy: 0.6920
Epoch 10/200
46/46 [==============================] - 0s 2ms/step - loss: 0.6468 - accuracy: 0.6878
Epoch 11/

46/46 [==============================] - 0s 2ms/step - loss: 0.5856 - accuracy: 0.7238
Epoch 84/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5855 - accuracy: 0.7231
Epoch 85/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5853 - accuracy: 0.7203
Epoch 86/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5853 - accuracy: 0.7224
Epoch 87/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5853 - accuracy: 0.7231
Epoch 88/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5852 - accuracy: 0.7217
Epoch 89/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5850 - accuracy: 0.7224
Epoch 90/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5849 - accuracy: 0.7224
Epoch 91/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5847 - accuracy: 0.7224
Epoch 92/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5847 - accuracy: 0.7203
Epoch 93/200


46/46 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7148
Epoch 165/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7162
Epoch 166/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7155
Epoch 167/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7141
Epoch 168/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7148
Epoch 169/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5801 - accuracy: 0.7155
Epoch 170/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5800 - accuracy: 0.7141
Epoch 171/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5800 - accuracy: 0.7134
Epoch 172/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5800 - accuracy: 0.7127
Epoch 173/200
46/46 [==============================] - 0s 2ms/step - loss: 0.5801 - accuracy: 0.7134
Epoc

# Prediction of testing data set

In [23]:
# Predict the classification of a new set of data
new_X_scaled = X_test_scaled
y_prediction=(nn_model.predict(new_X_scaled) > 0.5).astype("int32")


In [24]:
# Create a Dataframe for predictions
pred_df = pd.DataFrame(data=y_prediction, columns=["Prediction"])


In [25]:
# Create a Dataframe for y_test
single_results_df = pd.DataFrame({"Actual": y_test}).reset_index(drop=True)


In [26]:
# Create a Dataframe merging testing and prediction
neuronal_pred_df = pd.merge(single_results_df, pred_df,left_index=True, right_index=True)
neuronal_pred_df

,Actual,Prediction
0,1,0
1,1,0
2,1,0
3,0,0
4,0,0
...,...,...
478,0,0
479,0,0
480,1,0
481,1,1


In [27]:
#  Create a confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_prediction)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,314,14
Actual 1,131,24


# Deep learning

In [28]:
# Define the model - deep neural net
number_input_features = len(X.columns)
hidden_nodes_layer1 = 4
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 4)                 60        
                                                                 
 dense_3 (Dense)             (None, 4)                 20        
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
fit_model = nn.fit(X_train,y_train,epochs=300)
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
46/46 [==============================] - 1s 2ms/step - loss: 0.6899 - accuracy: 0.6802
Epoch 2/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.6802
Epoch 3/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6761 - accuracy: 0.6802
Epoch 4/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6704 - accuracy: 0.6802
Epoch 5/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6652 - accuracy: 0.6802
Epoch 6/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6606 - accuracy: 0.6802
Epoch 7/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6564 - accuracy: 0.6802
Epoch 8/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6527 - accuracy: 0.6802
Epoch 9/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.6802
Epoch 10/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6466 - accuracy: 0.6802
Epoch 11/

46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 83/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 84/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 85/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 86/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 87/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 88/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 89/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 90/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 91/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 92/300


46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 164/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 165/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 166/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 167/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 168/300
46/46 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 169/300
46/46 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 170/300
46/46 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 171/300
46/46 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 172/300
46/46 [==============================] - 0s 3ms/step - loss: 0.6267 - accuracy: 0.6802
Epoc

46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 245/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 246/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 247/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 248/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 249/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 250/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 251/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 252/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoch 253/300
46/46 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6802
Epoc

In [31]:
# Predict the classification of a new set of data
new_X_scaled = X_test_scaled
y_prediction=(nn.predict(new_X_scaled) > 0.5).astype("int32")


In [32]:
# Create a Dataframe for predictions
pred_df = pd.DataFrame(data=y_prediction, columns=["Prediction"])
# Create a Dataframe for y_test
single_results_df = pd.DataFrame({"Actual": y_test}).reset_index(drop=True)
# Create a Dataframe merging testing and prediction
neuronal_pred_df = pd.merge(single_results_df, pred_df,left_index=True, right_index=True)
neuronal_pred_df

,Actual,Prediction
0,1,0
1,1,0
2,1,0
3,0,0
4,0,0
...,...,...
478,0,0
479,0,0
480,1,0
481,1,0


In [33]:
#  Create a confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_prediction)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,328,0
Actual 1,155,0


In [34]:
X_test.head(50)

,Price,Pages,Language_Dummy_English,Language_Dummy_Other,Size_Dummy_Big-Publisher,Size_Dummy_Mid-Publisher,Size_Dummy_Small-Publisher,Famous_Dummy_High-Fame,Famous_Dummy_Low-Fame,Famous_Dummy_Mid-Fame,Categories_Dummy_Other,Categories_Dummy_Top_Category,Serie_Dummy_Other,Serie_Dummy_Serie
1857,17.16,336,1,0,1,0,0,1,0,0,0,1,1,0
1084,4.66,240,1,0,0,1,0,0,0,1,1,0,0,1
1661,3.74,288,1,0,1,0,0,1,0,0,0,1,0,1
1796,5.05,608,1,0,1,0,0,0,0,1,0,1,0,1
1756,5.74,94,1,0,1,0,0,0,1,0,0,1,1,0
1748,19.28,416,1,0,0,0,1,0,1,0,0,1,1,0
1289,12.19,304,1,0,1,0,0,0,0,1,0,1,0,1
1546,3.12,228,1,0,0,0,1,0,1,0,0,1,1,0
887,2.86,738,1,0,1,0,0,0,1,0,0,1,1,0
621,3.80,337,1,0,1,0,0,1,0,0,0,1,0,1


In [35]:
y_test


1857    1
1084    1
1661    1
1796    0
1756    0
       ..
1693    0
176     0
1088    1
899     1
629     1
Name: Rating_Classification, Length: 483, dtype: int64

In [36]:
df.to_csv('test.csv',index=False)